In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
import sys
import os
import re
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from sklearn.metrics import f1_score

In [ ]:
from lib import util, embeds, fitter, masker, features, sentiment, translate

In [ ]:
datapath = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data'
multilingual_model = 'distiluse-base-multilingual-cased-v1'

In [ ]:
frames = util.load_csv_dataframes(datapath)

In [ ]:
zdf = frames['train_zero_shot.csv']
odf = frames['train_one_shot.csv']
ddf = frames['dev.csv']
ddf_gold = frames['dev_gold.csv']
edf = frames['eval.csv']

In [ ]:
z_emb_multi = embeds.get_embeddings(zdf, modelname=multilingual_model, append=['MWE'])
d_emb_multi = embeds.get_embeddings(ddf, modelname=multilingual_model, append=['MWE'])
e_emb_multi = embeds.get_embeddings(edf, modelname=multilingual_model, append=['MWE'])

In [ ]:
z_score, z_probs, z_results = fitter.get_fit_results(z_emb_multi, zdf['Label'], d_emb_multi, ddf_gold['Label'])

In [ ]:
z_score

### Reload stored data

In [ ]:
zdf_bt3 = pd.read_pickle('data/zdf_bt3_20220104_1.pkl')
ddf_bt3 = pd.read_pickle('data/ddf_bt3_20220104_1.pkl')
edf_bt3 = pd.read_pickle('data/edf_bt3_20220105_1.pkl')

In [ ]:
edf_sub = frames['eval_submission_format.csv']

In [ ]:
# dropcols = ['Top score', 'FS', 'SS', 'Quotes', 'MWEdiff']
# dropcols = ['Hassub', 'FS', 'Nextdiff']
dropcols = ['MWEdiff', 'FS']
# dropcols = ['Top score 1', 'FS', 'MWEdiff']
zdf_t4 = fitter.get_trainable(zdf_bt3).drop(dropcols, axis=1)
ddf_t4 = fitter.get_trainable(ddf_bt3).drop(dropcols, axis=1)
ddf5_feat_score, ddf5_feat_probs, ddf5_feat_results = fitter.get_fit_results(zdf_t4, zdf['Label'], ddf_t4, ddf_gold['Label'])

In [ ]:
mup = fitter.multi_results(ddf_bt3, ddf_gold, z_results, z_probs, ddf5_feat_results, ddf5_feat_probs, ['Caps', 'Hassub'],['Quotes'])
f1_score(mup['Prediction'], mup['Label'], average='macro')

### Results from BERT model

##### Multilingual model for both EN,PT

In [ ]:
ddf_bert0 = pd.read_csv('models/ZeroShot/0/eval-dev/test_results_None.txt', sep='\t')
ddf_bert0_probs = pd.read_csv('models/ZeroShot/0/eval-dev/test_results_None.txt.probs', sep='\t')

In [ ]:
f1_score(ddf_bert0['prediction'].astype('str'), ddf_gold['Label'], average='macro')

##### English model for English, multilingual model for PT

In [ ]:
ddf_bert1 = pd.read_csv('models/ZeroShot/1/eval-dev/test_results_None.txt', sep='\t')
ddf_bert1_probs = pd.read_csv('models/ZeroShot/1/eval-dev/test_results_None.txt.probs', sep='\t')

In [ ]:
ddf_bert_comb = ddf_bert0.copy()
ddf_bert_comb_probs = ddf_bert0_probs.copy()
for idx,row in ddf_bert1.iterrows():
    _idx,_pred = row
    ddf_bert_comb.loc[_idx,'prediction'] = _pred
for idx,row in ddf_bert1_probs.iterrows():
    _idx,_pred = row
    ddf_bert_comb_probs.loc[_idx,'prediction'] = _pred

#### Combine BERT with feature model

In [ ]:
mup0 = fitter.multi_results(ddf_bt3, ddf_gold, ddf_bert0['prediction'].astype('str'), ddf_bert0_probs['probs'],
                            ddf5_feat_results, ddf5_feat_probs,
                            ['Caps', 'Hassub'],
                            ['Quotes'], agreeonly=True)
f1_score(mup0['Prediction'], mup0['Label'], average='macro')

##### Combine BERT (en+pt) with feature model

In [ ]:
mup1 = fitter.multi_results(ddf_bt3, ddf_gold, ddf_bert_comb['prediction'].astype('str'),
                            ddf_bert_comb_probs['probs'], ddf5_feat_results, ddf5_feat_probs,
                            ['Caps', 'Hassub'],
                            ['Quotes'], agreeonly=True)
f1_score(mup1['Prediction'], mup1['Label'], average='macro')

#### Eval results

In [ ]:
edf_bert0 = pd.read_csv('models/ZeroShot/0/eval-eval/test_results_None.txt', sep='\t')
edf_bert0_probs = pd.read_csv('models/ZeroShot/0/eval-eval/test_results_None.txt.probs', sep='\t')

In [ ]:
ez_score, ez_probs, ez_results = fitter.get_fit_results(z_emb_multi, zdf['Label'], e_emb_multi)

In [ ]:
dropcols3 = ['Top score', 'Hassub', 'Short', 'Nextdiff', 'MWEdiff']
zdf_et3 = fitter.get_trainable(zdf_bt3).drop(dropcols3, axis=1)
edf_et3 = fitter.get_trainable(edf_bt3).drop(dropcols3, axis=1)

edf_feat_score3, edf_feat_probs3, edf_feat_results3 = fitter.get_fit_results(zdf_et3, zdf['Label'], edf_et3)
edf_comb3 = fitter.multi_results(edf_bt3, None, ez_results, ez_probs, edf_feat_results3, edf_feat_probs3, ['Caps','Hassub'],['Quotes','!Trans'])
edf_comb3.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
edf_comb_bert1 = fitter.multi_results(edf_bt3, None, edf_bert0['prediction'].astype('str'), edf_bert0_probs['probs'], edf_feat_results3, edf_feat_probs3, ['Caps','Hassub'],['Quotes','!Trans'])

In [ ]:
edf_res_bert1 = edf_sub.copy()
edf_res_bert1.loc[edf_res_bert1['Setting'] == 'zero_shot', 'Label'] = edf_comb_bert1['Prediction']
edf_res_bert1.to_csv('data/eval_sub_20220103_2.csv', index=False)

In [ ]:
edf_comb_bert1[edf_comb_bert1['Pred1'] != edf_comb_bert1['Pred2']][['ID', 'MWE', 'Target', 'Trans', 'Caps', 'Hassub', 'Pred1', 'Pred2',
                                                                    'Prediction', 'PredictionTrans', 'PredictionQuotes', 'PredictionCaps', 'PredictionHassub']][50:100]

In [ ]:
edf_comb_bert2 = fitter.multi_results(edf_bt3, None, edf_bert0['prediction'].astype('str'), edf_bert0_probs['probs'], edf_feat_results3, edf_feat_probs3, ['Caps','Hassub'],['Quotes'])

In [ ]:
edf_res_bert2 = edf_sub.copy()
edf_res_bert2.loc[edf_res_bert1['Setting'] == 'zero_shot', 'Label'] = edf_comb_bert2['Prediction']
edf_res_bert2.to_csv('data/eval_sub_20220103_3.csv', index=False)

In [ ]:
edf_comb_bert2[edf_comb_bert2['Pred1'] != edf_comb_bert2['Pred2']][['ID', 'MWE', 'Target', 'Trans', 'Caps', 'Hassub', 'Pred1', 'Pred2',
                                                                    'Prediction', 'PredictionQuotes', 'PredictionCaps', 'PredictionHassub']][50:100]

This model finally gets a better result than baseline (71.6% vs 70.2%). This essentially means:
 - use the prediction if models agree
 - if they disagree
   - use the BERT base model (because of confidence) except when boolean features Quotes, Caps or Hassub say otherwise.
   - Trans feature (idiomatic interpretation if Trans==False) didn't turn out to be useful.

In [ ]:
edf_bert1 = pd.read_csv('models/ZeroShot/1/eval-eval/test_results_None.txt', sep='\t')
edf_bert1_probs = pd.read_csv('models/ZeroShot/1/eval-eval/test_results_None.txt.probs', sep='\t')

In [ ]:
edf_bert_comb = edf_bert0.copy()
edf_bert_comb_probs = edf_bert0_probs.copy()
for idx,row in edf_bert1.iterrows():
    _idx,_pred = row
    edf_bert_comb.loc[_idx,'prediction'] = _pred
for idx,row in edf_bert1_probs.iterrows():
    _idx,_pred = row
    edf_bert_comb_probs.loc[_idx,'prediction'] = _pred

In [ ]:
edf_comb_bert3 = fitter.multi_results(edf_bt3, None, edf_bert_comb['prediction'].astype('str'), edf_bert_comb_probs['probs'], edf_feat_results3, edf_feat_probs3, ['Caps','Hassub'],['Quotes'])

In [ ]:
edf_res_bert3 = edf_sub.copy()
edf_res_bert3.loc[edf_res_bert3['Setting'] == 'zero_shot', 'Label'] = edf_comb_bert3['Prediction']
edf_res_bert3.to_csv('data/eval_sub_20220103_4.csv', index=False)

In [ ]:
edf_comb_bert3[edf_comb_bert3['Prediction'] != edf_comb_bert2['Prediction']]

Surprisingly, using the English BERT for English only improves the result a tiny bit (71.66 > 71.62)